# Clean Strava & Weather Data
* Wrangle Strava JSON data into a pandas dataframe 
* Refine and create features from Strava data
* Clean data scraped from Weather Underground
* Join cleansed Strava & WUnderground data sets and write to file

In [196]:
import pickle
import pandas as pd

In [218]:
data_fpath = '/Users/bkhurley/git/diablo_velo/data/'
rawdata_fpath = data_fpath + 'raw_data/'
cleandata_fpath = data_fpath + 'clean_data/'

## Load JSON data

In [220]:
# load effort data
segment_efforts = pickle.load(open(rawdata_fpath + "all_efforts_656860.p", "rb"))

# load segment data (data on the segment itself, not the efforts)
segment_data = pickle.load(open(rawdata_fpath + "segment_656860.p", "rb"))
segment_data = segment_data

## Restructure data from JSON to dataframe
Let's reformat the data from a JSON object to a pandas dataframe, which is more suitable for analysis.

As we go, keep only features that may be useful to the analysis.

In [221]:
# features to keep
features = ['id', 'activity', 'athlete', 'elapsed_time', 'moving_time', 'start_date',
            'start_date_local', 'distance', 'start_index', 'end_index', 'device_watts', 
            'average_watts', 'average_heartrate', 'max_heartrate', 'average_cadence', 
            'kom_rank', 'pr_rank']

I parse the JSON data by iterating through each effort, and within each effort iterating through each attribute of the effort. Each of the JSON object's (retained) dictionary keys become columns, each row listing the value of those column variables for a given effort.

In [222]:
# initiate empty list to hold rows of data
effort_data = []

# now, iterate through each effort
for effort in segment_efforts:    
    row_array = []
    # within each effort, iterate through each variable
    for i in range(len(features)):
        # only process features of interest
        attribute = features[i]
        if attribute not in effort.keys():
            row_array.append(None)
        else:
            # 'activity' and 'athlete' keys contain dicts. Keep only id from those dicts
            if attribute in ['activity', 'athlete']:
                if type(effort[attribute]) == dict:
                    effort[attribute] = effort[attribute]['id']
            row_array.append(effort[attribute])
    
    # append row to data list 
    effort_data.append(row_array)

# convert data list to pandas dataframe
strava_data = pd.DataFrame(effort_data, columns=features)

Let's take a look at the new dataframe

In [223]:
strava_data.head()

,id,activity,athlete,elapsed_time,moving_time,start_date,start_date_local,distance,start_index,end_index,device_watts,average_watts,average_heartrate,max_heartrate,average_cadence,kom_rank,pr_rank
0,9131432,111986,5287,4578,4310,2006-04-23T16:06:17Z,2006-04-23T16:06:17Z,15183.9,450,1052,False,NaN,149.5,164.0,NaN,NaN,NaN
1,9131428,125030,1885,5627,5160,2006-05-06T15:40:44Z,2006-05-06T15:40:44Z,16799.1,327,1095,False,212.9,175.1,186.0,66.1,NaN,NaN
2,1670259583,82155037,1885,5494,5255,2006-06-09T15:50:57Z,2006-06-09T08:50:57Z,16920.2,112,931,False,201.6,165.7,175.0,67.9,NaN,1.0
3,24297526436,991534814,3719,5320,5320,2006-07-04T16:59:01Z,2006-07-04T09:59:01Z,15467.1,149,350,False,182.8,NaN,NaN,NaN,NaN,NaN
4,1670314655,82158071,1885,5153,5033,2006-07-29T15:30:41Z,2006-07-29T08:30:41Z,16898.2,272,1079,False,226.6,172.7,180.0,73.9,NaN,1.0


## Clean the data
Now, I will clean up the data by removing unwanted columns, refining some features, and generating new features.

### Remove columns that we probably won't need
We will go with the local start date, and the start and stop indices don't appear to be very informative to our needs.

In [224]:
del strava_data['start_date']
del strava_data['start_index']
del strava_data['end_index']

### Clean up dates & times

In [225]:
# convert start date to pandas' datetime format
strava_data['start_date_local'] = pd.to_datetime(strava_data['start_date_local'])

### Generate some extra features from the data we have

#### Date & time features
I'll break out the timestamp into various date and time features I can easily index later.

In [226]:
strava_data['date'] = strava_data['start_date_local'].apply(lambda x: x.date())
strava_data['time'] = strava_data['start_date_local'].apply(lambda x: x.time())
strava_data['day'] = strava_data['start_date_local'].apply(lambda x: x.weekday_name)
strava_data['hour'] = strava_data['start_date_local'].apply(lambda x: x.hour)
strava_data['month'] = strava_data['start_date_local'].apply(lambda x: x.month)
strava_data['year'] = strava_data['start_date_local'].apply(lambda x: x.year)

#### Performance features
Next, I'll work on features related to athletes' cycling performance.

In [227]:
# Strava logs distance in meters. Let's convert this to miles.
strava_data['distance (mi)'] = strava_data['distance'] * 0.00062137
# calculate miles per hour (Strava logs moving time in seconds)
strava_data['mph'] = strava_data['distance (mi)'] / ((strava_data['moving_time']/60)/60)

In [228]:
# calculate stopping time to see whether/how long riders took breaks
strava_data['time_stopped'] = strava_data['elapsed_time'] - strava_data['moving_time']

Let's peak at the data after cleaning up Strava features

In [229]:
strava_data.head()

,id,activity,athlete,elapsed_time,moving_time,start_date_local,distance,device_watts,average_watts,average_heartrate,...,pr_rank,date,time,day,hour,month,year,distance (mi),mph,time_stopped
0,9131432,111986,5287,4578,4310,2006-04-23 16:06:17,15183.9,False,NaN,149.5,...,NaN,2006-04-23,16:06:17,Sunday,16,4,2006,9.434820,7.880592,268
1,9131428,125030,1885,5627,5160,2006-05-06 15:40:44,16799.1,False,212.9,175.1,...,NaN,2006-05-06,15:40:44,Saturday,15,5,2006,10.438457,7.282644,467
2,1670259583,82155037,1885,5494,5255,2006-06-09 08:50:57,16920.2,False,201.6,165.7,...,1.0,2006-06-09,08:50:57,Friday,8,6,2006,10.513705,7.202538,239
3,24297526436,991534814,3719,5320,5320,2006-07-04 09:59:01,15467.1,False,182.8,NaN,...,NaN,2006-07-04,09:59:01,Tuesday,9,7,2006,9.610792,6.503543,0
4,1670314655,82158071,1885,5153,5033,2006-07-29 08:30:41,16898.2,False,226.6,172.7,...,1.0,2006-07-29,08:30:41,Saturday,8,7,2006,10.500035,7.510456,120


## Load weather history data and merge with Strava data
Load the weather history data I scraped from wunderground.com

In [230]:
weather = pd.read_csv(rawdata_fpath + 'wu_weather_history.csv')

In [231]:
weather.head()

,timestamp,Mean Temperature,Max Temperature,Min Temperature,Precipitation,Wind Speed,Max Wind Speed
0,20060101,52.0,62.0,43.0,0.02,6.0,20.0
1,20060102,52.0,54.0,48.0,0.41,5.0,14.0
2,20060103,50.0,55.0,46.0,0.00,6.0,16.0
3,20060104,55.0,60.0,50.0,0.00,6.0,12.0
4,20060105,52.0,59.0,46.0,0.00,6.0,12.0


timestamp is in a weird format. Let's transform it to a format that pandas' datetime method will recognize, and then transform it to a datetime object under a new column.

In [232]:
# convert timestamp to valid date representation
weather['date'] = weather['timestamp'].apply(
    lambda x: '-'.join([str(x)[:4], str(x)[4:6], str(x)[6:]])
)
# remove timestamp column
del weather['timestamp']

# store under new 'date' column, and put it in datetime.date format to match the date format in the Strava data set
weather['date'] = pd.to_datetime(weather['date'], format='%Y-%m-%d', errors='coerce')
weather['date'] = weather['date'].apply(lambda x: x.date())

Now, join the Strava and Weather Underground data sets, joining on 'date'

In [233]:
merged_data = pd.merge(strava_data, weather, on='date')

In [234]:
merged_data.head()

,id,activity,athlete,elapsed_time,moving_time,start_date_local,distance,device_watts,average_watts,average_heartrate,...,year,distance (mi),mph,time_stopped,Mean Temperature,Max Temperature,Min Temperature,Precipitation,Wind Speed,Max Wind Speed
0,9131432,111986,5287,4578,4310,2006-04-23 16:06:17,15183.9,False,NaN,149.5,...,2006,9.434820,7.880592,268,58.0,64.0,53.0,0.0,8.0,16.0
1,9131428,125030,1885,5627,5160,2006-05-06 15:40:44,16799.1,False,212.9,175.1,...,2006,10.438457,7.282644,467,58.0,68.0,51.0,0.0,15.0,17.0
2,1670259583,82155037,1885,5494,5255,2006-06-09 08:50:57,16920.2,False,201.6,165.7,...,2006,10.513705,7.202538,239,68.0,82.0,55.0,0.0,14.0,18.0
3,24297526436,991534814,3719,5320,5320,2006-07-04 09:59:01,15467.1,False,182.8,NaN,...,2006,9.610792,6.503543,0,70.0,84.0,55.0,0.0,14.0,20.0
4,1670314655,82158071,1885,5153,5033,2006-07-29 08:30:41,16898.2,False,226.6,172.7,...,2006,10.500035,7.510456,120,69.0,79.0,60.0,0.0,11.0,18.0


## Output cleaned data
Data look good! Now let's write the joined, cleaned data to a CSV file.

In [235]:
merged_data.to_csv(cleandata_fpath + 'strava_wu_data.csv', index=False)